In [1]:
import nltk
import re
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from wordcloud import WordCloud, STOPWORDS

In [2]:
#Downloading nltk StopWords and Wordnet 
nltk.download('stopwords')
nltk.download('wordnet')

Data_true = pd.read_csv("True.csv")
Data_fake = pd.read_csv("Fake.csv")

Data_true.nunique()
Data_fake.nunique()

[nltk_data] Downloading package stopwords to /Users/mason/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mason/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


title      17903
text       17455
subject        6
date        1681
dtype: int64

In [3]:
def wordpre(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

In [4]:
Data_true['label']= 1
Data_fake['label']= 0
Data = pd.concat([Data_true, Data_fake])
Data["Article"] = Data["title"] + Data["text"]
Data.sample(frac = 1) #Shuffle 100%
Data = Data.loc[:,['Article','label']]

In [5]:
##  Applying the wordpre method to the dataset
Data['Article']=Data['Article'].apply(wordpre)

Data.head()

,Article,label
0,as u s budget fight looms republicans flip t...,1
1,u s military to accept transgender recruits o...,1
2,senior u s republican senator let mr muell...,1
3,fbi russia probe helped by australian diplomat...,1
4,trump wants postal service to charge much mor...,1


In [6]:
x_train,x_test,y_train,y_test = train_test_split(Data['Article'], Data['label'], test_size=0.2, random_state=42)


In [7]:
x_train.shape


(35918,)

In [8]:
x_test.shape


(8980,)

In [9]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [10]:
#Stochastic Gradient Descent
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', SGDClassifier())])
SGDmodel = pipe.fit(x_train, y_train)
prediction = SGDmodel.predict(x_train)
print("Training accuracy: {}%".format(round(accuracy_score(y_train, prediction)*100,2)))
prediction = SGDmodel.predict(x_test)
print("Testing accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
SDGmodel_accuracy = round(accuracy_score(y_test, prediction)*100,2)

Training accuracy: 99.49%
Testing accuracy: 99.24%
